Q4. Campaign Impact Analysis 
○ What are the key performance indicators (KPIs) for assessing the success of
marketing campaigns?
○ Objective: Develop a framework for measuring campaign success using data
such as engagement rate, conversion rate, and customer lifetime value (CLV).
○ Expected Outcome: Propose actionable insights on how to improve marketing
Strategies.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
df_campaigns = pd.read_csv("../Data DSA3101/campaigns.csv")
df_engagement = pd.read_csv("../Data DSA3101/customer_engagement.csv")

In [4]:
print(df_campaigns.head())

   campaign_id     customer_segment  \
0            1              Retired   
1            2        Middle-Market   
2            3              Retired   
3            4     Budget-Conscious   
4            5  Young Professionals   

                                    campaign_name             campaign_type  \
0          Wealth Management Campaign for Retired                     Email   
1            Auto Loan Campaign for Middle-Market  Mobile App Notifications   
2                   Mortgage Campaign for Retired                     Email   
3     Personal Loan Campaign for Budget-Conscious                     Email   
4  Personal Loan Campaign for Young Professionals  Mobile App Notifications   

   start_date    end_date  recommended_product_id recommended_product_name  \
0  2022-03-17  2022-05-04                     107        Wealth Management   
1  2022-03-23  2022-05-02                     106                Auto Loan   
2  2022-05-22  2022-06-26                     103       

In [ ]:
print(df_engagement.head())

   engagement_id  customer_id  campaign_id channel sent delivered opened  \
0              1         3110            1   Email  Yes       Yes     No   
1              2         2468            1   Email  Yes       Yes     No   
2              3         7168            1   Email  Yes       Yes     No   
3              4         9616            1   Email  Yes       Yes     No   
4              5         7365            1   Email  Yes       Yes     No   

  clicked      engagement_date engagement_time conversion_status  
0      No  2022-03-26 14:34:53        14:34:53                No  
1      No  2022-03-29 06:49:52        06:49:52               Yes  
2      No  2022-04-02 13:34:07        13:34:07                No  
3      No  2022-03-26 11:50:42        11:50:42                No  
4     Yes  2022-04-05 14:36:54        14:36:54                No  


In [ ]:
df = pd.merge(df_campaigns,df_engagement,on="campaign_id")

In [ ]:
print(df.head())

   campaign_id customer_segment                           campaign_name  \
0            1          Retired  Wealth Management Campaign for Retired   
1            1          Retired  Wealth Management Campaign for Retired   
2            1          Retired  Wealth Management Campaign for Retired   
3            1          Retired  Wealth Management Campaign for Retired   
4            1          Retired  Wealth Management Campaign for Retired   

  campaign_type  start_date    end_date  recommended_product_id  \
0         Email  2022-03-17  2022-05-04                     107   
1         Email  2022-03-17  2022-05-04                     107   
2         Email  2022-03-17  2022-05-04                     107   
3         Email  2022-03-17  2022-05-04                     107   
4         Email  2022-03-17  2022-05-04                     107   

  recommended_product_name  campaign_budget  total_campaign_cost  ...  \
0        Wealth Management         58055.93             70597.33  ...   


In [ ]:
conversion_rate_name = df.groupby('campaign_name')['conversion_status'].apply(
    lambda x: (x == 'Yes').sum() *100/ len(x)
).reset_index()

conversion_rate_name.columns = ['campaign_name', 'conversion_rate']

In [ ]:
conversion_rate_type = df.groupby('campaign_type')['conversion_status'].apply(
    lambda x: (x == 'Yes').sum() *100/ len(x)
).reset_index()

conversion_rate_type.columns = ['campaign_type', 'conversion_rate']

In [ ]:
print(conversion_rate_name)

                                     campaign_name  conversion_rate
0          Auto Loan Campaign for Budget-Conscious         7.686454
1             Auto Loan Campaign for Middle-Market         7.726269
2       Auto Loan Campaign for Young Professionals         8.847185
3        Credit Card Campaign for Budget-Conscious         9.365559
4           Credit Card Campaign for Middle-Market         9.360878
5     Credit Card Campaign for Young Professionals         8.153078
6       Investment Product Campaign for High-Value        10.721248
7                    Mortgage Campaign for Retired         9.299896
8      Personal Loan Campaign for Budget-Conscious         8.238095
9         Personal Loan Campaign for Middle-Market         9.318498
10  Personal Loan Campaign for Young Professionals         7.414522
11      Savings Account Campaign for Middle-Market        11.953042
12            Savings Account Campaign for Retired        10.276297
13       Wealth Management Campaign for High-Val

In [ ]:
print(conversion_rate_type)

              campaign_type  conversion_rate
0               Direct Mail         6.827084
1                     Email        11.266883
2  Mobile App Notifications         8.241230
3                       SMS         7.438133
